# Loan Assistant Neural Core Workflow

## 1. Imports

In [1]:
import sys
from pathlib import Path
import torch
from models.arena import Arena
from models.dataset import DataFrameDataset
from models.neural_network import BaseLoanNN
from torch.optim import Adam
import torch.nn as nn
from logging import getLogger
from tools.log_controller import LogController

/home/emdablju/Documents/university/SUML/loan_assistant/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Setup Logging

In [2]:
log = getLogger(__name__)

log_controller = LogController(config_path=Path('config/logging_config.json'), dirs=Path('logs'))
log_controller.start()

log.info("Welcome to Loan Assistant Neural Core!")

2025-11-10 12:36:54,562 - INFO - __main__:6 - Welcome to Loan Assistant Neural Core!


## 3. Load Dataset

In [3]:
dataset_path = Path('repo/loan_data.csv')
log.info(f"Loading dataset from: {dataset_path}")
dataset = DataFrameDataset.from_csv(dataset_path, target_column='loan_status')
log.info("Dataset loaded successfully.")

2025-11-10 12:36:54,574 - INFO - __main__:2 - Loading dataset from: repo/loan_data.csv

2025-11-10 12:36:54,732 - INFO - __main__:4 - Dataset loaded successfully.


## 4. Hyperparameter Tuning

In [4]:
# log.info("Starting hyperparameter tuning...")
def objective(trial):
        input_size = len(dataset.feature_columns)
        num_hidden_layers = trial.suggest_int('num_hidden_layers', 1, 5)
        
        hidden_layers = nn.ModuleList()
        last_dim = input_size
        for i in range(num_hidden_layers):
            hidden_dim = trial.suggest_int(f'hidden_dim_{i}', 32, 128)
            hidden_layers.append(nn.Linear(last_dim, hidden_dim))
            last_dim = hidden_dim
        
        lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
        epochs = trial.suggest_int('epochs', 10, 50)
        
        model = BaseLoanNN(input_size=input_size, hidden_layers=hidden_layers, output_size=1)
        optimizer = Adam(model.parameters(), lr=lr)
        criterion = nn.BCEWithLogitsLoss()

        arena = Arena(model, optimizer, criterion, dataset, enable_validation=True)
        arena.train(epochs=epochs, batch_size=32)
        
        return arena.average_val_bce
    
# best_params, best_value = Arena.tune_hyperparameters(
#     objective=objective,
#     n_trials=3
# )
# log.info("Hyperparameter tuning finished.")

## 5. Initialize Model with Best Hyperparameters

In [5]:
log.info("Initializing model with best hyperparameters...")
input_size = len(dataset.feature_columns)
    
model = BaseLoanNN(input_size=13, hidden_layers=nn.ModuleList([nn.Linear(13, 64)]), output_size=1)
optimizer = Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()
log.info("Model initialized.")

2025-11-10 12:36:54,756 - INFO - __main__:1 - Initializing model with best hyperparameters...
2025-11-10 12:36:55,994 - INFO - __main__:7 - Model initialized.


## 6. Train the Final Model

In [6]:
arena = Arena(model, optimizer, criterion, dataset, enable_validation=True)

log.info("Starting final training with best hyperparameters...")
arena.train(epochs=40, batch_size=32)
log.info("Training finished.")

2025-11-10 12:36:56,004 - INFO - models.arena:15 - Using device: cpu
2025-11-10 12:36:56,140 - INFO - __main__:3 - Starting final training with best hyperparameters...
2025-11-10 12:36:56,780 - INFO - models.arena:55 - Epoch [1/40], BCE: 0.4390
2025-11-10 12:36:56,812 - INFO - models.arena:68 - Validation BCE: 0.3120
2025-11-10 12:36:57,328 - INFO - models.arena:55 - Epoch [2/40], BCE: 0.2809
2025-11-10 12:36:57,368 - INFO - models.arena:68 - Validation BCE: 0.2730
2025-11-10 12:36:57,794 - INFO - models.arena:55 - Epoch [3/40], BCE: 0.2637
2025-11-10 12:36:57,830 - INFO - models.arena:68 - Validation BCE: 0.2637
2025-11-10 12:36:58,273 - INFO - models.arena:55 - Epoch [4/40], BCE: 0.2587
2025-11-10 12:36:58,305 - INFO - models.arena:68 - Validation BCE: 0.2664
2025-11-10 12:36:58,680 - INFO - models.arena:55 - Epoch [5/40], BCE: 0.2554
2025-11-10 12:36:58,710 - INFO - models.arena:68 - Validation BCE: 0.2592
2025-11-10 12:36:59,077 - INFO - models.arena:55 - Epoch [6/40], BCE: 0.2532


## 7. Evaluate the Model

In [7]:
log.info("Starting evaluation...")
arena.evaluate(batch_size=32)
log.info("Evaluation finished.")

2025-11-10 12:37:12,181 - INFO - __main__:1 - Starting evaluation...
2025-11-10 12:37:12,233 - INFO - __main__:3 - Evaluation finished.


## 8. Save Results

In [9]:
# report_path = Path('logs/report.txt')
# log.info(f"Saving report to: {report_path}")
# with open(report_path, 'a') as f:
#     f.write("\n=-=-=-=-=-=-=-=-=-=-=-=-=-=\n")
#     f.write("Best Hyperparameters:\n")
#     for key, value in best_params.items():
#         f.write(f"  {key}: {value}\n")
#     f.write(f"Best Validation BCE: {best_value:.4f}\n\n")
#     f.write("Final Evaluation Results:\n")
#     f.write(arena.results)
#     f.write("\n=-=-=-=-=-=-=-=-=-=-=-=-=-=\n")
    
# log.info(f"Report saved successfully.")

# 9. Save model

In [14]:
model_path = Path('../app/agent/modules/model.pth')
log.info(f"Saving final model to: {model_path}")
torch.save(model.state_dict(), model_path)
log.info("Model saved successfully.")

2025-11-10 12:40:19,654 - INFO - __main__:2 - Saving final model to: ../app/agent/modules/model.pth
2025-11-10 12:40:19,657 - INFO - __main__:4 - Model saved successfully.


# 10. Load model

In [15]:
# 10. Load model
model = BaseLoanNN(input_size=13, hidden_layers=nn.ModuleList([nn.Linear(13, 64)]), output_size=1)
model.load_state_dict(torch.load('../app/agent/modules/model.pth'))  

<All keys matched successfully>

In [ ]:
predictions = model(dataset.features[:5])
correct = dataset.target[:5]

In [ ]:
# Apply sigmoid to get probabilities
probabilities = torch.sigmoid(predictions)
print("Predicted % probabilities for the first 5 samples:")
print(probabilities)
print("Correct labels for the first 5 samples:")
print(correct)

Predicted % probabilities for the first 5 samples:
tensor([[1.4308e-07],
        [3.6891e-01],
        [7.1944e-01],
        [7.6919e-01],
        [2.0563e-01]], grad_fn=<SigmoidBackward0>)
Correct labels for the first 5 samples:
tensor([0., 1., 1., 1., 0.])
